In [1]:
%%sh

apt-get install -y unzip

wget https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip 

unzip -o bank-additional.zip

Reading package lists...
Building dependency tree...
Reading state information...
unzip is already the newest version (6.0-23+deb10u1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
Archive:  bank-additional.zip
   creating: bank-additional/
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: bank-additional/bank-additional-full.csv  


--2020-09-04 16:24:42--  https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
Resolving sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)... 52.218.196.145
Connecting to sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)|52.218.196.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 432828 (423K) [application/zip]
Saving to: ‘bank-additional.zip’

     0K .......... .......... .......... .......... .......... 11% 1.18M 0s
    50K .......... .......... .......... .......... .......... 23% 1015K 0s
   100K .......... .......... .......... .......... .......... 35% 1021K 0s
   150K .......... .......... .......... .......... .......... 47% 64.4M 0s
   200K .......... .......... .......... .......... .......... 59% 47.2M 0s
   250K .......... .......... .......... ...

In [2]:
import pandas as pd
data = pd.read_csv('./bank-additional/bank-additional-full.csv')
data[:10]

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
5,45,services,married,basic.9y,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
6,59,admin.,married,professional.course,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
7,41,blue-collar,married,unknown,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
8,24,technician,single,professional.course,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
9,25,services,single,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [3]:
import sagemaker

prefix = 'sagemaker/tutorial-autopilot/input'
sess   = sagemaker.Session()

uri = sess.upload_data(path="./bank-additional/bank-additional-full.csv", key_prefix=prefix)
print(uri)


s3://sagemaker-us-east-2-916869366758/sagemaker/tutorial-autopilot/input/bank-additional-full.csv


In [5]:
import boto3, sys
ep_name = 'bdu-test-deploy-01'
sm_rt = boto3.Session().client('runtime.sagemaker')
tn=tp=fn=fp=count=0
with open('bank-additional/bank-additional-full.csv') as f:
    lines = f.readlines()
    for l in lines[1:2000]:   # Skip header
        l = l.split(',')           # Split CSV line into features
        label = l[-1]           # Store 'yes'/'no' label
        l = l[:-1]                # Remove label
        l = ','.join(l)           # Rebuild CSV line without label

        response = sm_rt.invoke_endpoint(EndpointName=ep_name, 
                                         ContentType='text/csv',       
                                         Accept='text/csv', Body=l)
        response = response['Body'].read().decode("utf-8")
        #print ("label %s response %s" %(label,response))
        if 'yes' in label:  # Sample is positive
            if 'yes' in response:  # True positive
                tp=tp+1
            else:  # False negative
                fn=fn+1
        else:  # Sample is negative
            if 'no' in response:
                # True negative
                tn=tn+1
            else:  # False positive
                fp=fp+1
        count = count+1
        if (count % 100 == 0):   
            sys.stdout.write(str(count)+' ')
            
print ("Done")

accuracy  = (tp+tn)/(tp+tn+fp+fn)
precision = tp/(tp+fp)
recall    = tn/(tn+fn)
f1        = (2*precision*recall)/(precision+recall)

print ("%.4f %.4f %.4f %.4f" % (accuracy, precision, recall, f1))

100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 Done
0.9830 0.5606 0.9974 0.7178


In [9]:
sess.delete_endpoint(endpoint_name=ep_name)

In [10]:
%%sh

aws s3 rm --recursive s3://sagemaker-us-east-2-916869366758/sagemaker/tutorial-autopilot/

delete: s3://sagemaker-us-east-2-916869366758/sagemaker/tutorial-autopilot/input/bank-additional-2500.csv
delete: s3://sagemaker-us-east-2-916869366758/sagemaker/tutorial-autopilot/output/test/data-processor-models/test-dpp2-1-485713a7999e49fcba8d6e3efe1c3395b51201a4672a41a689d/output/model.tar.gz
delete: s3://sagemaker-us-east-2-916869366758/sagemaker/tutorial-autopilot/output/test/data-processor-models/test-dpp0-1-118ce910a26b4341ba130d1c0a0b43fd719de2b31fca4873be7/output/model.tar.gz
delete: s3://sagemaker-us-east-2-916869366758/sagemaker/tutorial-autopilot/output/test/data-processor-models/test-dpp4-1-a830e33fc2b04019ac76ad2564ffb1dffa5a9bff9ef84f81804/output/model.tar.gz
delete: s3://sagemaker-us-east-2-916869366758/sagemaker/tutorial-autopilot/output/
delete: s3://sagemaker-us-east-2-916869366758/sagemaker/tutorial-autopilot/input/bank-additional-full.csv
delete: s3://sagemaker-us-east-2-916869366758/sagemaker/tutorial-autopilot/output/test/data-processor-models/test-dpp1-1-ad4f7